In [1]:
import numpy as np
from scipy.spatial.transform import Rotation

from numpy.typing import ArrayLike

## Measurement

In [2]:
class Measurement(object):
    # Choose right vector (arbitrary but stable)
    # use world +Y as up, so right = up × forward
    UP = np.asarray([0, 0, 1])

    @staticmethod
    def to_rotation(vec: ArrayLike) -> Rotation:
        """
        Derives a quaternion for a coordinate system: 
        Forward=[1,0,0], Left=[0,1,0], Up=[0,0,1].
        The result will have zero roll relative to the XY plane.
        """
        vec = np.asarray(vec)
        
        # 1. Normalize the target forward vector (Local X)
        forward = vec / np.linalg.norm(vec)
        
        # 2. Derive the Local Left vector (Local Y)
        # Crossing Global Up with Forward gives a vector on the XY plane
        left = np.cross(Measurement.UP, forward)
        
        # Handle singularity: if target is straight up/down, cross product is zero
        if np.linalg.norm(left) < 1e-6:
            # Default to global Left [0, 1, 0] to resolve ambiguity
            left = np.array([0, 1, 0])
        else:
            left /= np.linalg.norm(left)
        
        # 3. Derive the Re-orthogonalized Local Up vector (Local Z)
        up = np.cross(forward, left)
        
        # 4. Construct Rotation Matrix
        # Columns correspond to Local X (Forward), Local Y (Left), and Local Z (Up)
        matrix = np.column_stack((forward, left, up))
    
        # 5. Convert to Quaternion [w, x, y, z]
        return Rotation.from_matrix(matrix)

In [3]:
r = Measurement.to_rotation([1, 0, 0])
assert np.allclose(r.as_euler('xyz', degrees=True), [0, 0, 0], atol=.01)

r = Measurement.to_rotation([0, 1, 0])
assert np.allclose(r.as_euler('xyz', degrees=True), [0, 0, 90], atol=.01)

r = Measurement.to_rotation([1, 1, 0])
assert np.allclose(r.as_euler('xyz', degrees=True), [0, 0, 45], atol=.01)

r = Measurement.to_rotation([0, 1, 1])
assert np.allclose(r.as_euler('xyz', degrees=True), [0, -45, 90], atol=.01)

r = Measurement.to_rotation([1, 1, 1])
assert np.allclose(r.as_euler('xyz', degrees=True), [0, -35.26, 45], atol=.01)

r = Measurement.to_rotation([-1, 0, 1])
assert np.allclose(r.as_euler('xyz', degrees=True), [0, -45, 180], atol=.01)

## Point3D

In [4]:
class Point3D(Measurement):
    # measurement coordinates are (0,0,range)
    range: float
    rotation: Rotation
    
    def __init__(self, vec: ArrayLike, forward=[1, 0, 0]):
        vec = np.asarray(vec)
        rng = np.linalg.norm(vec)
        self.range = float(rng)
        
        vec = vec / self.range
        self.rotation = Measurement.to_rotation(vec)

In [5]:
p = Point3D([1, 1, 1])
assert np.allclose(p.range, 1.73, atol=.01)
assert np.allclose(p.rotation.as_euler('xyz', degrees=True), [0, -35.26, 45], atol=.01)